# Part 2, Topic 1: Introduction to Voltage Glitching (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Similarly to clock glitching, inserting brief glitches into the power line of an embedded device can result in skipped instructions and corrupted results. Besides providing a more reliable glitch on some targets when compared to clock glitching, voltage glitching also has the advanatage that the Vcc pins on chips are always accessable. This won't be covered in this course, but it can also be used to glitch a device asynchronous to its clock.*

**LEARNING OUTCOMES:**

* Understanding voltage glitch settings
* Building a voltage glitch and crash map.
* Modifying glitch circuit to increase glitch success

## Voltage Glitch Hardware

The ChipWhisperer uses the same hardware block for both voltage and clock glitching, with the only difference being where the glitch output is routed to. Instead of routing to HS2, voltage glitching is performed by routing the glitch to either the `glitch_hp` transistor or the `glitch_lp` transistor. This can be done via the following API calls:

```python
scope.io.glitch_hp = True #enable HP glitch
scope.io.glitch_hp = False #disable LP glitch
scope.io.glitch_lp = True #enable LP glitch
scope.io.glitch_lp = False #disable LP glitch
```

While the hardware block are the same, you'll need to change how it's configued. You wouldn't want to try routing `"clock_xor"` to the glitch transistor and oscillate Vcc like the device's clock! Instead, the following two output settings are best suited to voltage glitching:

1. `"glitch_only"` - insert a glitch for a portion of a clock cycle based on `scope.glitch.width` and `scope.glitch.offset`
1. `"enable_only"` - insert a glitch for an entire clock cycle

Typically, the `"enable_only"` setting will be too powerful for most devices. One situation where it outshines `"glitch_only"` is in glitching asychronous to the target's clock. An example of this is glitching a target with an internal clock. In this case, the ChipWhisperer's clock can be boosted far above the target's to insert a precise glitch, with `repeat` functioning as `width` and `ext_offset` functioning as `offset`.

### Voltage Glitching vs. Clock Glitching

Voltage glitching has some obvious benefits over clock glitching, such as working for a wider varitey of targets, but its downsides are less obvious. One of the biggest is how much it depends on the actual glitch circuit itself. With clock glitching, it's relatively easy to insert a glitch - there's nothing external trying to keep the clock at a certain voltage level. This is very different for a target's power pins. When we try to drop the power pin to ground, there's a lot of stuff fighting us to keep the power pin at the correct voltage, such as decoupling capacitors, bulk supply capacitors, and the voltage regulator supplying the voltage. This means when we make small changes to the glitch circuit, the glitch settings and even our ability to insert a glitch at all completely change! Consider glitching a target on the CW308 UFO board. If you switch your coaxial cable length from 20cm to 40 cm, you'll need to find entirely new glitch settings to repeat the attack (if it's still even possible). This is quite easy to see on an oscilloscope or  using the ChipWhisperer's ADC: longer cables and lower valued shunt resistors will make the glitch less sharp and increase ringing.

While your first thought might be to go for as sharp a glitch as possible, this often won't result in a high glitch success rate. If you're unable to find any working glitches with your current setup, it might be worth changing you hardware setup a bit. For example, on the ChipWhisperer Lite 1 part, you can desolder SJ5 and solder header pins to JP6. Even just connecting these pins with a jumper will have different glitch behaviour than with a soldered SJ5.

You can refer to the training slides for more information about finding good glitch settings, as well as more on the theory side of voltage glitching.

### The Lab

To introduce you to volatge glitching and find some settings, we're going to walk back through the clock glitching loop lab. You may want to capture some power traces while you're first experimenting with glitches to see what effects different glitch widths have on the power trace. Another thing to keep in mind is that targets often won't tolerate the Vcc pin dropping for an extended period of time without crashing - once you see the target start to crash, you won't see much else with larger widths.

One thing you might have to change is the glitch repeat value. Depending on how wide your glitch is, the voltage at the power pin may not recover by the time the next glitch is inserted. This can have to effect of increasing subsequent glitches' strength, which may or may not be desirable. Since glitches inserted with repeat > 1 have different strength, it's a good idea to scan through ext_offset as well.

###  Higher Frequency Glitching

The XMEGA target, and to a lesser extent the STM32F3, is very difficult to glitch with the default ChipWhisperer settings. Try bumping the clock frequency to 24MHz for the STM32 or 32MHz for the XMEGA and use a repeat 5-10 with both the high power and low power glitches active. You'll need to adjust the baud rate by the same proportion as the clock. This will increase the glitch precision, as the glitch width/offset step is based on the target clock, and may decrease the stability of the target.

Another setup that seems to work with the XMEGA is SJ5 unsoldered, JP6 jumpered, high+low power glitch, 32MHz, and repeat=5.

The ChipWhisperer Husky, with its PLL based glitching system, can typically glitch targets without needing to change the target's frequency.

### Disabling Logging

When glitching (or just running normally in earlier labs), you may have seen various warnings from loggers ChipWhisperer uses. This often has useful information, especially if things don't work right, but for voltage 
glitching especially, it mostly clutters up any print output you have. As such, we'll disable logging for the voltage glitching labs:

```python
cw.set_all_log_levels(cw.logging.CRITICAL)
```

You can reenable logging via

```python
cw.set_all_log_levels(cw.logging.WARNING)
```

In [1]:
import serial
import threading
import time
from collections import deque
import re
from typing import List, Optional, Dict, Any

class SerialLineBuffer:
    def __init__(self, port="/dev/ttyACM2", baud=115200, max_lines=10000):
        self.port = port
        self.baud = baud
        self.max_lines = max_lines

        self._buf = deque(maxlen=max_lines)  # stores decoded lines (str)
        self._lock = threading.Lock()

        self._thread = None
        self._running = False

    # ---------- Thread control ----------
    def start(self):
        if self._running:
            return
        self._running = True
        self._thread = threading.Thread(target=self._worker, daemon=True)
        self._thread.start()

    def stop(self):
        self._running = False
        if self._thread:
            self._thread.join()

    # ---------- Worker ----------
    def _worker(self):
        try:
            ser = serial.Serial(self.port, self.baud, timeout=0.2)
        except Exception as e:
            print(f"[SerialLineBuffer] Error opening serial: {e}")
            self._running = False
            return

        while self._running:
            try:
                raw = ser.readline()
                if not raw:
                    continue
                line = raw.decode(errors="ignore").rstrip("\r\n")
                if not line:
                    continue
                with self._lock:
                    self._buf.append(line)
            except Exception as e:
                print(f"[SerialLineBuffer] Read error: {e}")
                time.sleep(0.2)

        ser.close()

    # ---------- Main-loop API ----------
    def pop_all(self):
        """Return all lines currently in the buffer and clear it."""
        with self._lock:
            lines = list(self._buf)
            self._buf.clear()
            return lines

    def get_all(self):
        """Non-destructive: return a snapshot of all lines."""
        with self._lock:
            return list(self._buf)

    def clear(self):
        """Just clear the buffer."""
        with self._lock:
            self._buf.clear()

            


# Very specific patterns based on logs

RE_CPU_RESET = re.compile(
    r'^NOTICE:\s+CPU:\s+STM32MP257FAI\s+Rev\.Y\b'
)

RE_CRASH_START = re.compile(
    r'^"(Synchronous Abort|Undefined Instruction|Exception)" handler,\s+esr\s+(0x[0-9A-Fa-f]{8})\s*$'
)

RE_CRASH_END = re.compile(
    r'^Code:\s+(?:[0-9A-Fa-f]{8}\s+)*\([0-9A-Fa-f]{8}\)\s*$'
)

# ESR anywhere in the crash block
RE_ESR = re.compile(
    r'\besr\s+(0x[0-9A-Fa-f]{8})\b'
)

# Runtime ELR/LR line (ignore the reloc one)
# Example: elr: 00000000880003c0 lr : 0000000088000370
RE_ELRLR = re.compile(
    r'^elr:\s*([0-9A-Fa-f]{16})\s+lr\s*:\s*([0-9A-Fa-f]{16})\s*$'
)


def _extract_esr(lines: List[str]) -> Optional[str]:
    for l in lines:
        m = RE_ESR.search(l)
        if m:
            return m.group(1)
    return None


def _extract_elr_lr(lines: List[str]):
    """Return (elr, lr) from the *runtime* line, skipping any '(reloc)' line."""
    for l in lines:
        if "(reloc)" in l:
            continue
        m = RE_ELRLR.match(l)
        if m:
            return m.group(1), m.group(2)
    return None, None


def _extract_code(lines: List[str]):
    """Return (prev_instrs, fault_instr) from the Code: line, or (None, None)."""
    for l in lines:
        if not l.startswith("Code:"):
            continue
        parts = re.findall(r"[0-9A-Fa-f]{8}", l)
        if not parts:
            return None, None
        prev = parts[:-1]
        fault = parts[-1]
        return prev, fault
    return None, None


def extract_crash_from_lines(lines: List[str]) -> Optional[Dict[str, Any]]:
    """
    Scan a list of log lines and return info about the *last* crash found.
    Returns None if no crash was detected.
    """
    crashes = []  # list of {"lines": [...], "complete": bool}
    crash_active = False
    crash_buf: List[str] = []

    for line in lines:
        # Hard reset: if we were in a crash, finalize as incomplete
        if RE_CPU_RESET.match(line):
            if crash_active and crash_buf:
                crashes.append({"lines": crash_buf[:], "complete": False})
                crash_buf = []
                crash_active = False
            # We don't need to track runs here; this is just crash extraction
            continue

        # Crash start
        if RE_CRASH_START.match(line):
            crash_active = True
            crash_buf = [line]
            continue

        # Inside a crash block
        if crash_active:
            crash_buf.append(line)
            if RE_CRASH_END.match(line):
                crashes.append({"lines": crash_buf[:], "complete": True})
                crash_buf = []
                crash_active = False
            continue

    # Crash started but never completed within this chunk → incomplete
    if crash_active and crash_buf:
        crashes.append({"lines": crash_buf[:], "complete": False})

    if not crashes:
        return None

    # Take the last crash in this block
    last = crashes[-1]
    raw = last["lines"]
    complete = last["complete"]

    esr = _extract_esr(raw)
    elr, lr = _extract_elr_lr(raw)
    prev_instrs, fault_instr = _extract_code(raw)

    return {
        "complete": complete,
        "esr": esr,
        "elr": elr,
        "lr": lr,
        "prev_instructions": prev_instrs,
        "faulting_instruction": fault_instr,
        "raw_dump": raw,
    }


In [2]:
import serial, time

DEVICE = '/dev/ttyACM1'  # or '/dev/gwinstek' after the udev rule + re-login

with serial.Serial(DEVICE, 115200, timeout=1) as s:
    s.dtr = True
    s.rts = False
    time.sleep(0.1)
    s.reset_input_buffer(); s.reset_output_buffer()
    s.write(b"*IDN?\r\n")
    time.sleep(0.2)
    print((s.read(1024) or b"").decode(errors="ignore").strip() or "<no response>")

def scope_single(device="/dev/ttyACM1"):
    s = serial.Serial(device, 115200, timeout=1)
    try:
        s.write(b":SING\r\n")
    finally:
        s.close()
        

import serial, time, struct, json, numpy as np, re
import matplotlib.pyplot as plt
from contextlib import contextmanager



_num = re.compile(r"[+-]?(?:\d+\.\d*|\d*\.\d+|\d+)(?:[eE][+-]?\d+)?")

@contextmanager
def open_port(dev):
    s = serial.Serial(dev, 115200, timeout=1)
    s.dtr = True; s.rts = False
    time.sleep(0.1)
    s.reset_input_buffer(); s.reset_output_buffer()
    try:
        yield s
    finally:
        s.close()

def w(s, cmd): s.write((cmd + "\n").encode())
def qf(s, cmd):
    w(s, cmd); time.sleep(0.05)
    txt = (s.read(256) or b"").decode(errors="ignore")
    m = _num.search(txt); return float(m.group(0)) if m else float("nan")

def read_block(s):
    while True:
        b = s.read(1)
        if not b: raise RuntimeError("Timeout waiting for '#'")
        if b == b'#': break
    nd = int(s.read(1).decode())
    L  = int(s.read(nd).decode())
    buf = bytearray()
    while len(buf) < L:
        chunk = s.read(L - len(buf))
        if not chunk: raise RuntimeError(f"Short read: {len(buf)}/{L}")
        buf += chunk
    return bytes(buf)

def grab_memory(s, ch, longmem=False):
    for c in (":ACQuire:STOP", ":STOP"):
        w(s, c); time.sleep(0.02); s.read(64)
    w(s, f":ACQuire{ch}:{'LMEMory?' if longmem else 'MEMory?'}"); time.sleep(0.07)
    p = read_block(s)
    dt_hdr = struct.unpack(">f", p[0:4])[0]   # BIG-endian float32
    chan   = p[4]
    raw = np.frombuffer(p[8:], dtype=">i2")   # samples are big-endian int16
    return dt_hdr, chan, raw

def scope2csv(CSV_FILE="capture.csv", DEVICE=None, USE_LONG_MEMORY=True, Y_LIMS=None):

    # Load calibration (volts = m*code + b)
    CAL = json.load(open("gds_cal_linear.json"))

    with open_port(DEVICE) as s:
        # Put both channels in DC coupling
        for ch in ("1","2"):
            w(s, f":CHANnel{ch}:COUPling 1"); time.sleep(0.03); s.read(48)

        # Acquire both channels
        dt1, ch1_id, raw1 = grab_memory(s, "1", USE_LONG_MEMORY)
        dt2, ch2_id, raw2 = grab_memory(s, "2", USE_LONG_MEMORY)

        # Optional status/info (not used for timing anymore)
        tdiv  = qf(s, ":TIMebase:SCALe?")
        vdiv1 = qf(s, ":CHANnel1:SCALe?")
        vdiv2 = qf(s, ":CHANnel2:SCALe?")
        coup1 = int(qf(s, ":CHANnel1:COUPling?"))
        coup2 = int(qf(s, ":CHANnel2:COUPling?"))

    # Align and build time using header dt (start at 0)
    N = min(len(raw1), len(raw2))
    raw1, raw2 = raw1[:N], raw2[:N]

    # Warn if the two headers disagree noticeably; still use CH1 dt
    if N > 0 and np.isfinite(dt1) and np.isfinite(dt2) and dt1 > 0 and dt2 > 0:
        rel = abs(dt1 - dt2) / max(dt1, 1e-30)
        if rel > 5e-3:
            print(f"⚠️  dt mismatch: CH1={dt1:.9g}s vs CH2={dt2:.9g}s  (rel={rel:.3e}); using CH1 dt.")
    dt_used = dt1
    t = np.arange(N, dtype=float) * dt_used

    # Apply calibration
    m1, b1 = CAL["1"]["m"], CAL["1"]["b"]
    m2, b2 = CAL["2"]["m"], CAL["2"]["b"]
    v1 = m1*raw1.astype(np.float64) + b1
    v2 = m2*raw2.astype(np.float64) + b2

    # Log
    print(f"dt(header): CH1={dt1:.9g}s  CH2={dt2:.9g}s  -> using dt={dt_used:.9g}s | N={N}")
    print(f"CH1(id={ch1_id}) coupling={coup1} V/div={vdiv1:.6g}  map: v={m1:.6g}*code+{b1:.6g}")
    print(f"CH2(id={ch2_id}) coupling={coup2} V/div={vdiv2:.6g}  map: v={m2:.6g}*code+{b2:.6g}")

    # === Save CSV exactly with this timebase ===
    out = np.column_stack([t, v1, v2])
    np.savetxt(CSV_FILE, out, delimiter=",", header="time_s,CH1_V,CH2_V", comments="")
    print(f"Saved {CSV_FILE} (t starts at 0, step=dt(header))")

    
# GDS waveform GUI — scroll, zoom, set timebase & volts/div, toggle channels, export PNG/CSV
# Expects CSV with columns: time_s, CH1_V, CH2_V
# Controls:
#  Timebase (s/div), Center time, Quick pan/zoom, Units (s/ms/µs/ns)
#  Volts/div, Vertical center, Autoscale Y toggle
#  Show/hide CH1 & CH2, Reset, Export PNG/CSV
#
# Requires: ipywidgets, matplotlib, numpy

import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import (
    HBox, VBox, Label, Checkbox, FloatText, BoundedFloatText, Button,
    Dropdown, FloatSlider, FloatRangeSlider, Text, Layout, Output
)
from IPython.display import display, clear_output

CSV_FILE="capture.csv"#placeholder

# ---------------- load data helpers ----------------
def load_csv(path):
    data = np.genfromtxt(path, delimiter=",", names=True)
    cols = list(data.dtype.names)
    if 'time_s' not in cols:
        raise ValueError("CSV must contain a 'time_s' column")
    t = data['time_s'].astype(float)
    v1 = data['CH1_V'].astype(float) if 'CH1_V' in cols else np.full_like(t, np.nan)
    v2 = data['CH2_V'].astype(float) if 'CH2_V' in cols else np.full_like(t, np.nan)
    return t, v1, v2

# ---------------- widgets ----------------
file_path = Text(value=CSV_FILE, description="CSV", layout=Layout(width="50%"))
btn_load  = Button(description="Load", tooltip="Load CSV")

# Channel visibility
ch1_on = Checkbox(value=True, description="CH1")
ch2_on = Checkbox(value=True, description="CH2")

# Time units
units = Dropdown(
    options=[("s",1.0), ("ms",1e-3), ("µs",1e-6), ("ns",1e-9)],
    value=1.0, description="Units"
)

# Timebase controls (10 horizontal divisions, just like the scope)
s_div = FloatText(value=1e-3, description="Time/div (s)", layout=Layout(width="200px"))
t_center = FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-12, description="Center t (s)",
                       readout_format=".6g", continuous_update=False, layout=Layout(width="95%"))

btn_pan_left  = Button(description="◀ Pan")
btn_pan_right = Button(description="Pan ▶")
btn_zoom_in   = Button(description="Zoom +")
btn_zoom_out  = Button(description="Zoom −")
btn_fit       = Button(description="Fit all")

# Vertical scaling controls (8 vertical divisions by default)
v_div = FloatText(value=1.0, description="Volts/div", layout=Layout(width="200px"))
v_center = FloatText(value=0.0, description="V center", layout=Layout(width="180px"))
auto_y = Checkbox(value=True, description="Auto Y")

# Export / reset
export_png = Button(description="Export PNG")
export_csv = Button(description="Export CSV")
btn_reset  = Button(description="Reset")

# Output areas
fig_out = Output()
status  = Output()

# ---------------- state ----------------
t = np.array([]); v1 = np.array([]); v2 = np.array([])
tmin = 0.0; tmax = 1.0
N_HDIV = 10  # horizontal divisions (scope convention)
N_VDIV = 8   # vertical divisions typical on many scopes; adjust if you prefer

def set_time_widgets_limits():
    global tmin, tmax
    if t.size == 0:
        tmin, tmax = 0.0, 1.0
    else:
        tmin, tmax = float(np.nanmin(t)), float(np.nanmax(t))
        if not np.isfinite(tmin) or not np.isfinite(tmax) or tmax <= tmin:
            tmin, tmax = 0.0, 1.0
    t_center.min = tmin
    t_center.max = tmax
    if not (tmin <= t_center.value <= tmax):
        t_center.value = (tmin + tmax)/2


def set_controls_from_window(t0, t1):
    """Set s/div and center from a desired [t0, t1] window."""
    width = max(t1 - t0, 1e-30)
    s_div.value = width / N_HDIV
    t_center.value = (t0 + t1) / 2

def time_window_from_controls():
    """Compute [t0, t1] from center + s/div, without clamping to data."""
    width = max(s_div.value * N_HDIV, 1e-30)
    tc = t_center.value
    t0 = tc - width/2
    t1 = tc + width/2
    return t0, t1

def pan(fraction=0.2):
    """Pan by a fraction of current window width."""
    t0, t1 = time_window_from_controls()
    width = t1 - t0
    shift = width * fraction
    set_controls_from_window(t0 + shift, t1 + shift)
    redraw()

def zoom(factor=0.5):
    """Zoom window width by factor (<1 zooms in, >1 zooms out) around current center."""
    t0, t1 = time_window_from_controls()
    tc = (t0 + t1) / 2
    half = (t1 - t0) * factor / 2
    set_controls_from_window(tc - half, tc + half)
    redraw()


def redraw(_=None):
    if t.size == 0:
        with fig_out:
            clear_output()
            print("No data loaded.")
        return

    # Compute time window and mask
    t0, t1 = time_window_from_controls()
    m = (t >= t0) & (t <= t1)
    if not np.any(m):
        with fig_out:
            clear_output()
            print("Time window empty.")
        return

    # Time units
    scale = units.value
    unit_label = [k for k, v in units.options if v == units.value][0]
    tx = t[m] / scale

    # Vertical scaling: either autoscale or volts/div around center
    if auto_y.value:
        y0 = np.nanmin([np.nanmin(v1[m]) if ch1_on.value else np.nan, np.nanmin(v2[m]) if ch2_on.value else np.nan])
        y1 = np.nanmax([np.nanmax(v1[m]) if ch1_on.value else np.nan, np.nanmax(v2[m]) if ch2_on.value else np.nan])
        if not np.isfinite(y0) or not np.isfinite(y1) or y1 <= y0:
            y0, y1 = -1.0, 1.0
    else:
        center = v_center.value
        half_span = v_div.value * N_VDIV / 2.0
        y0, y1 = center - half_span, center + half_span

    with fig_out:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(9, 5))
        if ch1_on.value:
            ax.plot(tx, v1[m], label="CH1")
        if ch2_on.value:
            ax.plot(tx, v2[m], label="CH2")
        ax.grid(True, which="both", alpha=0.4)
        ax.minorticks_on()
        ax.set_xlim(tx[0], tx[-1])
        ax.set_ylim(y0, y1)
        ax.set_xlabel(f"Time ({unit_label})")
        ax.set_ylabel("Voltage (V)")
        handles, labels = ax.get_legend_handles_labels()
        if handles:
            ax.legend(loc="best")
        # draw division grid hints (optional): 10 vertical lines for time divisions
        try:
            import matplotlib.ticker as mticker
            # Time divisions
            td = s_div.value / scale
            ax.xaxis.set_major_locator(mticker.MultipleLocator(td))
            # Volts divisions (if manual)
            if not auto_y.value and v_div.value > 0:
                ax.yaxis.set_major_locator(mticker.MultipleLocator(v_div.value))
        except Exception:
            pass
        plt.show()

    # Update status
    with status:
        clear_output()
        print(f"Window: {t0:.6g}s .. {t1:.6g}s | span={t1-t0:.6g}s  |  Time/div={s_div.value:.6g}s  |  Units={unit_label}")
        if auto_y.value:
            print(f"Y: autoscale -> [{y0:.6g}, {y1:.6g}] V")
        else:
            print(f"Y: center={v_center.value:.6g} V, Volts/div={v_div.value:.6g} V  (N_VDIV={N_VDIV})")

def load_data(_=None):
    global t, v1, v2
    with status:
        clear_output()
        print("Loading:", file_path.value)
    t, v1, v2 = load_csv(file_path.value)
    set_time_widgets_limits()
    # Initialize to entire record width
    set_controls_from_window(tmin, tmax)
    # Reasonable initial vertical center as median of available channels
    avail = []
    if np.isfinite(np.nanmedian(v1)): avail.append(np.nanmedian(v1))
    if np.isfinite(np.nanmedian(v2)): avail.append(np.nanmedian(v2))
    if avail:
        v_center.value = float(np.nanmedian(avail))
    redraw()

def do_export_png(_=None):
    if t.size == 0:
        with status:
            clear_output(); print("Nothing to export.")
        return
    t0, t1 = time_window_from_controls()
    m = (t >= t0) & (t <= t1)
    scale = units.value; unit_label = [k for k, v in units.options if v == units.value][0]
    tx = t[m] / scale

    # y limits same logic as redraw
    if auto_y.value:
        y0 = np.nanmin([np.nanmin(v1[m]) if ch1_on.value else np.nan, np.nanmin(v2[m]) if ch2_on.value else np.nan])
        y1 = np.nanmax([np.nanmax(v1[m]) if ch1_on.value else np.nan, np.nanmax(v2[m]) if ch2_on.value else np.nan])
        if not np.isfinite(y0) or not np.isfinite(y1) or y1 <= y0:
            y0, y1 = -1.0, 1.0
    else:
        center = v_center.value
        half_span = v_div.value * N_VDIV / 2.0
        y0, y1 = center - half_span, center + half_span

    fig, ax = plt.subplots(figsize=(9, 5))
    if ch1_on.value:
        ax.plot(tx, v1[m], label="CH1")
    if ch2_on.value:
        ax.plot(tx, v2[m], label="CH2")
    ax.grid(True, which="both", alpha=0.4); ax.minorticks_on()
    ax.set_xlim(tx[0], tx[-1]); ax.set_ylim(y0, y1)
    ax.set_xlabel(f"Time ({unit_label})"); ax.set_ylabel("Voltage (V)")
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(loc="best")
    # add division locators
    try:
        import matplotlib.ticker as mticker
        td = s_div.value / scale
        ax.xaxis.set_major_locator(mticker.MultipleLocator(td))
        if not auto_y.value and v_div.value > 0:
            ax.yaxis.set_major_locator(mticker.MultipleLocator(v_div.value))
    except Exception:
        pass
    fig.savefig("gds_view.png", dpi=150, bbox_inches="tight")
    plt.close(fig)
    with status:
        clear_output(); print("Saved PNG -> gds_view.png")

def do_export_csv(_=None):
    if t.size == 0:
        with status:
            clear_output(); print("Nothing to export.")
        return
    t0, t1 = time_window_from_controls()
    m = (t >= t0) & (t <= t1)
    if not np.any(m):
        with status:
            clear_output(); print("Time window empty; CSV not written.")
        return
    arr = [t[m]]; hdr = ["time_s"]
    if ch1_on.value:
        arr.append(v1[m]); hdr.append("CH1_V")
    if ch2_on.value:
        arr.append(v2[m]); hdr.append("CH2_V")
    out_arr = np.column_stack(arr)
    np.savetxt("gds_view_slice.csv", out_arr, delimiter=",", header=",".join(hdr), comments="")
    with status:
        clear_output(); print("Saved CSV -> gds_view_slice.csv")

# ---------------- wiring ----------------
btn_load.on_click(load_data)
units.observe(redraw, names="value")
s_div.observe(redraw, names="value")
t_center.observe(redraw, names="value")
v_div.observe(redraw, names="value")
v_center.observe(redraw, names="value")
auto_y.observe(lambda ch: (setattr(ymin_dummy,'value',0)), names="value")  # dummy to trigger
ch1_on.observe(redraw, names="value")
ch2_on.observe(redraw, names="value")
btn_pan_left.on_click(lambda _: pan(-0.2))
btn_pan_right.on_click(lambda _: pan(+0.2))
btn_zoom_in.on_click(lambda _: zoom(0.5))
btn_zoom_out.on_click(lambda _: zoom(2.0))
btn_fit.on_click(lambda _: (set_controls_from_window(tmin, tmax), redraw()))
btn_reset.on_click(lambda _: load_data())
export_png.on_click(do_export_png)
export_csv.on_click(do_export_csv)

# manual-Y widgets for autoscale toggle (no direct inputs, just kept for completeness)
ymin_dummy = FloatText(value=0.0, layout=Layout(width="1px", height="1px", visibility="hidden"))

# ---------------- layout ----------------
row_top = HBox([file_path, btn_load, units, btn_reset])
row_time = VBox([
    HBox([s_div, btn_pan_left, btn_pan_right, btn_zoom_in, btn_zoom_out, btn_fit]),
    t_center
])
row_channels = HBox([Label("Channels:"), ch1_on, ch2_on])
row_y = HBox([auto_y, v_div, v_center, export_png, export_csv])

ui = VBox([row_top, row_time, row_channels, row_y, fig_out, status])

def show(csv_path):
    file_path.value = csv_path
    display(ui)
    load_data()



GW,GDS-1152A,EJ161479,V1.08


In [ ]:
scope2csv(CSV_FILE="capture1.csv", DEVICE=DEVICE, USE_LONG_MEMORY=True, Y_LIMS=None)


In [ ]:
show("capture1.csv")

In [3]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWHUSKY'
allowable_exceptions = None

#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")


prog = cw.programmers.STM32FProgrammer


import time
time.sleep(0.05)
scope.default_setup()





INFO: Found ChipWhisperer😍
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.clock.adc_rate                     changed from 0.0                       to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx                
scope.io.hs2                             changed from None                      to clkgen            

In [4]:

# Set up UART
target.baud = 115200

##set trigger out on mcx connector

scope.io.glitch_trig_mcx = "glitch"
#scope.io.glitch_trig_mcx = "trigger"


    
scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 200e6
scope.clock.adc_mul = 1
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4" # Trigger on rising edge of IO4
scope.glitch.enabled = True
scope.glitch.clk_src = "pll"
scope.clock.pll.update_fpga_vco(600e6)
scope.glitch.output = 'enable_only' #apply crowbar for one entire husky cycle, glitch only doesnt seem to work
scope.glitch.trigger_src = "ext_single" #fire only once everytime trigger is tripped
#cannot use ext continuous if relying on target signals cause the glitcher will trigger itself again

scope.glitch.offset = 1000
scope.glitch.width = 1000
scope.glitch.ext_offset = 25# offset from trigger in husky cycles 
scope.glitch.repeat = 500#crowbar active time in husky cycles
scope.io.glitch_lp = False
scope.io.glitch_hp = True
#for async glitching, crank the clock speed all the way up to 200mhz, can try overclocking also too
#for some reason the glitch width changes depending on whether the LP mos is active or not
#HP only: 500 clock cycles (repeat) ->3.440uS
#LP only: 500 clock cycles (repeat) ->3.940uS
#BOTH: 500 clock cycles (repeat) ->3.940uS
#choose HP mos only for best timing accuracy (scope.glitch.repeat/scope.clock.clkgen_freq)
#offset from trigger = (scope.glitch.ext_offset/scope.clock.clkgen_freq) + some fixed prop delay (100+ ns)

scope.adc.lo_gain_errors_disabled = True
scope.adc.clip_errors_disabled = True
scope.io.nrst = 'high_z'
scope.arm()             # Arm the glitch + ADC modules
#print(scope.glitch)
#print(scope.adc)
#print(scope.clock)


'''  
def reboot_flush():
    scope.io.nrst = 'low'
    time.sleep(0.05)   # hold reset low
    scope.glitch.repeat = 500
    scope.glitch.trigger_src = "manual"
    scope.arm()
    print(scope.errors)
    scope.errors.clear()
    scope.glitch.trigger_src = "ext_single" 
    target.flush()
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)   # let target start running
'''
    
def reboot_flush(expected_msg="s", timeout=10.0):
    scope.io.nrst = 'low'
    time.sleep(0.05)   # hold reset low
    target.flush()
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)   # let target start running
    
    start_time = time.time()
    buffer = ""

    while time.time() - start_time < timeout:
        data = target.read(100, timeout=0.01)
        if data:
            buffer += data
            if expected_msg in buffer:
                #print(f"[reboot_flush] Detected startup message: '{expected_msg}'")
                return True
        time.sleep(0.01)

    #print(f"[reboot_flush] Timeout waiting for '{expected_msg}' (got: '{buffer.strip()}')")
    return False

(ChipWhisperer Scope WARNING|File ChipWhispererHuskyClock.py:525) ADC frequency must be between 1MHz and 300000000.0MHz - ADC mul has been adjusted to 1


In [ ]:
#The husky USB comms stack is VERY buggy, if it doesnt work either wait for the timeout or just restart the entire notebook
scope.sc._timeout = 50  # milliseconds
#ext_offset is useless here, since manually triggered
#Uses the trig out port to trigger external oscilloscope everytime the glitch is fired, no trigger present in manual mode
#Very nice UI with knobs and sliders to try out manual asynchronous glitching

import ipywidgets as widgets
from IPython.display import display
import threading
import time

scope.io.glitch_trig_mcx = "glitch"

# ------------------------------
# UI widgets
# ------------------------------
repeat_slider = widgets.IntSlider(description='Repeat', min=600, max=1000, value=600)
offset_slider = widgets.IntSlider(description='Ext Offset', min=0, max=500, value=25)
freq_slider = widgets.FloatLogSlider(  # log slider so low + high rates are usable
    description='Freq (Hz)', base=10, min=0, max=3, step=0.01,
    value=100.0, readout_format='.2f'
)
attack_button = widgets.ToggleButton(description="Start Attack", value=False)

# Display UI
display(repeat_slider, offset_slider, freq_slider, attack_button)

# ------------------------------
# Glitch parameter updates
# ------------------------------
def update_glitch_settings(change=None):
    scope.glitch.repeat = repeat_slider.value
    scope.glitch.ext_offset = offset_slider.value

repeat_slider.observe(update_glitch_settings, names='value')
offset_slider.observe(update_glitch_settings, names='value')
update_glitch_settings()  # apply initial values

# ------------------------------
# Frequency control
# ------------------------------
_pulser_period_s = 1.0 / freq_slider.value

def _update_period_from_slider(change=None):
    global _pulser_period_s
    freq = max(0.1, float(freq_slider.value))  # avoid div-by-zero
    _pulser_period_s = 1.0 / freq

freq_slider.observe(_update_period_from_slider, names='value')
_update_period_from_slider()

# ------------------------------
# Manual glitch trigger function
# ------------------------------
def _fire_one_glitch():
    try:
        scope.glitch.manual_trigger()
        return
    except AttributeError:
        pass
    try:
        scope.io.trigger_now()
        return
    except AttributeError:
        pass
    raise RuntimeError("No manual trigger function found. Replace _fire_one_glitch().")

# ------------------------------
# Pulser thread logic
# ------------------------------
_pulser_thread = None
_pulser_stop = threading.Event()

def _glitch_pulser():
    next_t = time.perf_counter()
    while not _pulser_stop.is_set():
        now = time.perf_counter()
        if now >= next_t:
            try:
                _fire_one_glitch()
            except Exception as e:
                print("Manual trigger error:", e)
            next_t += _pulser_period_s
            if next_t < now - 5 * _pulser_period_s:
                next_t = now + _pulser_period_s
        time.sleep(min(0.001, _pulser_period_s / 10.0))

def _start_pulser():
    global _pulser_thread
    if _pulser_thread and _pulser_thread.is_alive():
        return
    _pulser_stop.clear()
    _pulser_thread = threading.Thread(target=_glitch_pulser, daemon=True)
    _pulser_thread.start()

def _stop_pulser():
    _pulser_stop.set()

# ------------------------------
# Attack toggle handler
# ------------------------------
def on_attack_toggle(change):
    if change["new"]:
        print("⚡ Glitch attack ENABLED")
        scope.glitch.trigger_src = "manual"
        update_glitch_settings()
        scope.glitch.enabled = True
        _start_pulser()
    else:
        print("🛑 Glitch attack DISABLED")
        _stop_pulser()
        scope.glitch.enabled = False
        scope.glitch.trigger_src = "ext_single"

attack_button.observe(on_attack_toggle, names="value")

# ------------------------------
# UART logger thread
# ------------------------------
def uart_logger():
    while True:
        try:
            while target.in_waiting():
                data = target.read()
                print(data, end='')  # already string
        except Exception as e:
            print("UART error:", e)
        time.sleep(0.1)

target.baud = 115200

if 'uart_thread' not in globals():
    uart_thread = threading.Thread(target=uart_logger, daemon=True)
    uart_thread.start()


In [ ]:
import logging

logging.basicConfig(
    filename="/home/user/Desktop/glitch_mainwhile_log.txt",
    filemode="a",          # append (use "w" to overwrite)
    level=logging.INFO,
    format="%(asctime)s | %(message)s",
    force=True             # <-- reconfigure even if already set (Jupyter!)
)


logging.info("EXPECTED VALUE: 'r9999', GLITCH VALUE: 'r0'")
logging.info("ACTUAL RESULTS (run id, glitch width, offset from trigger, return value, crash data)")

buf = SerialLineBuffer("/dev/ttyACM2", 115200)
buf.start()
reboot_flush()

# === Prediction-band sweep: offset ≈ m*repeat + b ± BAND_HALF ===
m = -1.0324358026302414
b = 1041.6720098017875
BAND_HALF = 13                 # ~95% pointwise prediction half-width (rounded)
REPEAT_MIN, REPEAT_MAX = 585, 680
#send g, wait for trigger, glitch, wait for result, check if result

gc = cw.GlitchController(groups=["success", "reset", "normal", "hang", "crash"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()

run_id = 0
num_tries = 10 # increase to get better glitch stats
gc.set_global_step([1]) # reduce to fine tune glitching
gc.set_range("ext_offset", -BAND_HALF, BAND_HALF)
gc.set_range("repeat", REPEAT_MIN, REPEAT_MAX)
gc.set_range("tries", 1, num_tries)
gc.set_step("repeat", 1)
#gc.set_step("ext_offset", 1)
gc.set_step("tries", 1)
scope.glitch.enabled = True

def log_result(outcome, val, crash):
    """
    outcome: 'success' | 'normal' | 'crash' | 'hang' | 'reset'
    val:     string returned from target (e.g. 'r0', 'r9999', '')
    crash:   dict from extract_crash_from_lines(...) or None
    """
    crash_info = crash or {}

    logging.info(
        "RUN(%d) repeat=%d ext_offset=%.3f ret='%s' outcome=%s "
        "crash_complete=%s esr=%s elr=%s lr=%s prev_instr=%s fault_instr=%s",
        run_id,
        scope.glitch.repeat,
        scope.glitch.ext_offset,
        val,
        outcome,
        crash_info.get("complete"),
        crash_info.get("esr"),
        crash_info.get("elr"),
        crash_info.get("lr"),
        crash_info.get("prev_instructions"),
        crash_info.get("faulting_instruction"),
    )


try:
    for glitch_setting in gc.glitch_values():
        run_id = run_id + 1
        scope_single()
        time.sleep(0.2)
        if scope.adc.state:
            #Device is slow to boot?
            reboot_flush()
            time.sleep(0.05)


        _repeat = glitch_setting[0]
        _ext_off = m*_repeat + b + glitch_setting[1]     

        scope.glitch.repeat = _repeat
        scope.glitch.ext_offset = _ext_off
        scope.arm()
        target.write("g")
        val = target.read(30, 100)#For loop check
        #print(val)
        normal = (val == "r9999") 
        glitch = (val == "r0")
        lines = buf.pop_all()
        crash = extract_crash_from_lines(lines)


        if glitch is True:
            gc.add("success")
            print((scope.glitch.repeat, scope.glitch.ext_offset, val, "YAYYYYY"))
            logging.info(f"({run_id},{scope.glitch.repeat}, {scope.glitch.ext_offset}, '{val}', success, ...complete with all the crash data)")
            #input("Press Enter to continue…")
            log_result("success", val, crash)
            reboot_flush()
            
        elif normal is True:
            gc.add("normal")
            #print((scope.glitch.repeat, scope.glitch.ext_offset, val, "normal"))
            #input("Press Enter to continue…")
            log_result("normal", val, crash)
            
        elif crash: 
            gc.add("crash")
            print("Crash found:")
            print("  complete:", crash["complete"])
            print("  esr     :", crash["esr"])
            print("  elr     :", crash["elr"])
            print("  lr      :", crash["lr"])
            print("  prev    :", crash["prev_instructions"])
            print("  fault   :", crash["faulting_instruction"])
            log_result("crash", val, crash)
            reboot_flush()#to replace with sleep and see if reset reason changes
            
        elif bool(scope.io.tio_states[3]):#trigger never went low, but no valid response, hang?
            gc.add("hang")
            print((scope.glitch.repeat, scope.glitch.ext_offset, val, "hang"))
            log_result("hang", val, crash)
            reboot_flush()
            
        else:
            gc.add("reset")
            print((scope.glitch.repeat, scope.glitch.ext_offset, val, "reset"))
            log_result("reset", val, crash)
            reboot_flush()

except KeyboardInterrupt:
    buf.stop()
    print("stopped buffer")

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

#add simple check to take a screenshot every 100th try
#glitch width, glitch offset, return, category, crash code if any, line match (single or range), screenshot if any

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='hang count:', disabled=True)

IntText(value=0, description='crash count:', disabled=True)

...

(588, 423, '', 'hang')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000758
  lr      : 0000000088000370
  prev    : ['7100081f', '54000061', '52800000', '140000cb']
  fault   : d503201f
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000400
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000640
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
(588, 423, '', 'hang')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 00000000880003c0
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000238
  lr      : 0000000088000ad0
  prev    : ['f9000fe0', 'f9000be1', 'f90007e2', 'f90027ff']
  fault   

Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000400
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000b80
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3c', '9c3955c3']
  fault   : 63c6aa3c
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000e80
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3c', '9c3955c3']
  fault   : 63c6aa3c
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000480
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
(588, 431, '', 'hang')
(588, 431, '', 'hang')
(588, 431, '', 'hang')
(588, 431, '', 'hang')
(588, 431, '', 'hang')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000e40
  lr      : 0000000088000370
  

Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 00000000880003c0
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
(588, 441, '', 'reset')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 00000000880003c0
  lr      : 0000000088000370
  prev    : ['d503201f', 'd503201f', 'd503201f', 'd503201f']
  fault   : d503201f
Crash found:
  complete: True
  esr     : 0xc6000022
  elr     : 0000000088000338
  lr      : 0000000088000ad0
  prev    : ['8b000020', '3900001f', 'f94027e0', '91000400']
  fault   : 910143ff
(588, 442, '', 'reset')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000f40
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3c', '9c3955c3']
  fault   : 63c6aa3c
(588, 442, '', 'reset')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000b20
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3

Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000e00
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3c', '9c3955c3']
  fault   : 63c6aa3c
(589, 423, '', 'reset')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000e00
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3c', '9c3955c3']
  fault   : 63c6aa3c
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000240
  lr      : 0000000088000ad0
  prev    : ['f90007e2', 'f90027ff', 'f94007e2', 'b202e7e0']
  fault   : f29999a0
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000760
  lr      : 0000000088000370
  prev    : ['52800000', '140000cb', 'd503201f', 'd503201f']
  fault   : d503201f
(589, 423, '', 'hang')
Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000760
  lr      : 0000000088000370
  prev    : ['52800000', '140000cb', 'd503201f', 'd503201f']
  fault  

Crash found:
  complete: True
  esr     : 0x02000000
  elr     : 0000000088000e40
  lr      : 0000000088000370
  prev    : ['63c6aa3c', '9c3955c3', '63c6aa3c', '9c3955c3']
  fault   : 63c6aa3c


In [ ]:
#send g, wait for trigger, glitch, wait for result, check if result
#100, ext offset from 420 to 530, repeat from 570 onwards

#conditional glitch check
#gc.set_step("ext_offset", 5)
#gc.set_range("width", 2400, 2700)
#gc.set_range("offset", 2400, 2700)

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


num_tries = 10 # increase to get better glitch stats
gc.set_global_step([1]) # reduce to fine tune glitching
    
#todo offset extoffset by repeat, eg if repeat 500 offset 500 if repeat 1000 offset 0
#gc.set_range("ext_offset", 950, 1050)
gc.set_range("ext_offset", 515, 540)
gc.set_range("repeat", 585, 680)
gc.set_range("tries", 1, num_tries)
gc.set_step("repeat", 1)
#gc.set_step("ext_offset", 1)
gc.set_step("tries", 1)
scope.glitch.enabled = True

_prev_repeat = None
_ext_shift = 0  # how much to subtract from ext_offset overall

for glitch_setting in gc.glitch_values():
    #scope_single()
    #time.sleep(1)
    if scope.adc.state:
        #Device is slow to boot?
        reboot_flush()
        time.sleep(0.05)
        
        
    _repeat = glitch_setting[0]
    if _prev_repeat is None:
        _prev_repeat = _repeat
    elif _repeat != _prev_repeat:
        # assumes repeat increases by +1 per step
        _ext_shift += 1
        _prev_repeat = _repeat
    # compute adjusted ext_offset (clamp at 0 just in case)
    _ext_off_raw = glitch_setting[1]
    _ext_off_adj = max(0, _ext_off_raw - _ext_shift)        

    scope.glitch.repeat = _repeat
    scope.glitch.ext_offset = _ext_off_adj 
    scope.arm()
    target.write("g")
    val = target.read(5, 50)#For loop check
    #print(val)
    normal = (val == "r9999") 
    #glitch = val.startswith("r") and val[1:].isdigit() and int(val[1:]) != 9999#112,offset 106
    glitch = (val == "r0")
#| (val == "r2634") | (val == "02291") wth

    
    if glitch is True:
        gc.add("success")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "YAYYYYY"))
        #input("Press Enter to continue…")
        reboot_flush()
    elif normal is True:
        gc.add("normal")
        #print((scope.glitch.repeat, scope.glitch.ext_offset, val, "normal"))
        #input("Press Enter to continue…")
    elif bool(scope.io.tio_states[3]):#trigger never went low, but no valid response, hang?
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "hang"))
        reboot_flush()
    else:
        gc.add("reset")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "reset"))
        reboot_flush()

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

In [ ]:
#LOOP GLITCH
#send g, wait for trigger, glitch, wait for result, check if result

#gc.set_step("ext_offset", 5)
#gc.set_range("width", 2400, 2700)
#gc.set_range("offset", 2400, 2700)
import logging

logging.basicConfig(
    filename="/home/user/Desktop/glitch_mainwhile_log.txt",
    filemode="a",          # append (use "w" to overwrite)
    level=logging.INFO,
    format="%(asctime)s | %(message)s",
    force=True             # <-- reconfigure even if already set (Jupyter!)
)



logging.info("EXPECTED VALUE: 'r9801'")
logging.info("ACTUAL RESULTS (glitch width, offset from trigger, return value)")

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


num_tries = 5 # increase to get better glitch stats
gc.set_global_step([1]) # reduce to fine tune glitching
    

gc.set_range("ext_offset", 2850, 8000)
gc.set_range("repeat", 680, 720)
gc.set_range("tries", 1, num_tries)
gc.set_step("repeat", 5)
gc.set_step("ext_offset", 10)
gc.set_step("tries", 1)
scope.glitch.enabled = True

for glitch_setting in gc.glitch_values():
    if scope.adc.state:
        #Device is slow to boot?
        reboot_flush()
        time.sleep(0.05)
        
   
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    scope.arm()
    target.write("g")
    val = target.read(15, 10).strip()#For loop check
    #print(val)
    normal = (val == "r9801")
    glitch = val.startswith("r") and val[1:].isdigit() and int(val[1:]) != 9801


    
    if glitch is True:
        gc.add("success")
        logging.info(f"({scope.glitch.repeat}, {scope.glitch.ext_offset}, '{val}')")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val))
    elif normal is True:
        gc.add("normal")
        logging.info(f"({scope.glitch.repeat}, {scope.glitch.ext_offset}, '{val}')")
        #print((scope.glitch.repeat, scope.glitch.ext_offset, val))
    else:
        gc.add("reset")
        logging.info(f"({scope.glitch.repeat}, {scope.glitch.ext_offset}, '{val}')")
        #print((scope.glitch.repeat, scope.glitch.ext_offset, val))
        reboot_flush()

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

In [ ]:
#send g, wait for trigger, glitch, wait for result, check if result
#infinite while loop glitch
#3 assumed probable scenarios : success(glitched out of while loop and prints r9999, 
#                               normal(program runs normally, trigger pin still high)
#                               reset (trigger pin goes down for whatever reason, BOR,RST, etc)
target.write("g")
scope.glitch.trigger_src = "ext_single"
prev_repeat = 0
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


num_tries = 1 # increase to get better glitch stats
gc.set_global_step([1]) # reduce to fine tune glitching
    

gc.set_range("ext_offset", 1792, 1892)
gc.set_range("repeat", 650, 1020)
gc.set_range("tries", 1, num_tries)
gc.set_step("repeat", 1)
#gc.set_step("ext_offset", 1)
gc.set_step("tries", 1)
scope.glitch.enabled = True

for glitch_setting in gc.glitch_values():
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    
    
    # --- RESET ON glitch width CHANGE ---
    if prev_repeat is not None and scope.glitch.repeat != prev_repeat:
        # Good-measure reset sequence
        reboot_flush()
        time.sleep(0.05)
        scope.glitch.trigger_src = "ext_single"
        scope.arm()
        target.write("g")
        time.sleep(0.05)
    prev_repeat = scope.glitch.repeat
    # -------------------------------------
    
    
    val = target.read(5, 10)#For loop check
    #print(val)
    normal = bool(scope.io.tio_states[3]) #by right high means program still running i think
    glitch = (val == "r9999")


    
    if glitch is True:#the program escapes and prints "r9999"
        gc.add("success")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "YAYYYYY"))
        
        
    elif normal is True: #trigger pin still high, assume program still running
        gc.add("normal")
        print((scope.glitch.repeat, val, "normal"))
        
        #program is still running
        scope.glitch.trigger_src = "manual"
        scope.arm()
        
    else:#trigger is now low, assume BOR kicked in and cpu is reset
        gc.add("reset")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "reset"))
        reboot_flush()# board should be booted by now?
        
        #use the regular trigger
        scope.glitch.trigger_src = "ext_single"
        scope.arm()
        target.write("g")
        

print("Done glitching")
#enable logging
#cw.set_all_log_levels(cw.logging.WARNING)
"""


"""

In [ ]:
#send g, wait for trigger, glitch, wait for result, check if result

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


num_tries = 5 # increase to get better glitch stats
gc.set_range("tries", 1, 1)
#gc.set_range("width", 2400, 2700)
#gc.set_range("offset", 2400, 2700)
gc.set_global_step([20]) # reduce to fine tune glitching
    
#gc.set_step("ext_offset", 5)
gc.set_range("ext_offset", 0, 8191)
gc.set_range("repeat", 5, 100)


for glitch_setting in gc.glitch_values():
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    time.sleep(0.01)
    

In [ ]:
scope.glitch.repeat = 150
scope.glitch.trigger_src = "manual"
scope.arm()
#print(scope.errors)
scope.errors.clear()
#scope.glitch.trigger_src = "ext_continuous" #fire everytime trigger is tripped

In [ ]:
import time

try:
    while True:
        target.write("g0000")
        time.sleep(0.01)
except KeyboardInterrupt:
    print("Loop stopped.")


In [ ]:

##set trigger out on mcx connector

#scope.io.glitch_trig_mcx = "glitch"
scope.io.glitch_trig_mcx = "trigger"


    
scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 10e6
scope.clock.adc_mul = 1

scope.adc.basic_mode = "rising_edge"

scope.trigger.triggers = "tio4" # Trigger on rising edge of IO4

scope.glitch.enabled = True
scope.glitch.clk_src = "pll"
scope.io.glitch_hp = True
scope.io.glitch_lp = False
    


In [ ]:
reboot_flush()

scope.glitch.ext_offset = 25
scope.glitch.repeat = 500

scope.arm()
target.write("g")
scope.capture()
val = target.read(5, 10)#For loop check
normal = (val == "r2500")

glitch = val.startswith("r") and val[1:].isdigit() and int(val[1:]) != 2500

print("Normal:", normal)
print("Glitch:", glitch)
print("Value:", val)

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, bufferlen=int(10E6))

Next, we'll setup the glitch. The main differences here compared to clock glitching is the use of `scope.io.glitch_lp/hp`, which activates the transistors used for voltage, and `scope.glitch.output = 'glitch_only'`, which sends the glitch output to the transistors instead of xoring it with the clock like with clock glitching.

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True
    scope.glitch.clk_src = "pll"
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
else:
    scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

Some tips for finding good glitches:

1. There's a lot of stuff fighting our glitch this time - unlike the clock line, the Vcc rail isn't supposed to oscillate! As such shorter glitches will have no effect. Often, good widths will be just below when the target starts consistantly crashing, so a good strategy is to find the minimum width where crashes always happen, then backing the width off a bit.
1. The repeat parameter behaves very differently than with voltage glitching - at the boosted clock rate, the Vcc often won't recover before the next glitch. Try different repeat values as well.
1. We've built in a success/reset measurement into the glitch loop. Once you've found some glitch spots, this will help you evaluate which ones are best for your target.

It can take a very long time to do go through the full search space, so you may want to stop after you get a certain number of succeses. By default here, it will be 1, but you may want to change it to 10, 20, or go even higher.

In [ ]:
MAX_SUCCESSES = 100000

num_tries = 5 # increase to get better glitch stats
gc.set_range("tries", 1, num_tries)
if scope._is_husky:
    gc.set_range("width", 2400, 2700)
    gc.set_range("offset", 2400, 2700)
    gc.set_global_step([10]) # reduce to fine tune glitching
    
    gc.set_step("ext_offset", 1)
    gc.set_range("ext_offset", 10, 50)
    scope.glitch.repeat = 1
    scope.adc.lo_gain_errors_disabled = True
    scope.adc.clip_errors_disabled = True
elif PLATFORM=="CWLITEXMEGA":
    gc.set_range("width", 43.5, 47.8)
    gc.set_range("offset", -48, -10)
    gc.set_range("ext_offset", 7, 10)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 11
elif PLATFORM == "CWLITEARM":
    #should also work for the bootloader memory dump
    gc.set_range("width", 34, 36)
    gc.set_range("offset", -40, 10)
    gc.set_range("ext_offset", 4, 30)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 7
elif PLATFORM == "CW308_STM32F3":
    #these specific settings seem to work well for some reason
    #also works for the bootloader memory dump
    gc.set_range("ext_offset", 4, 30)
    gc.set_range("width", 47.6, 49.6)
    gc.set_range("offset", -19, -21.5)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 5

gc.set_step("tries", 1)

In [ ]:
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

scope.adc.timeout = 0.5

reboot_flush()
loff = scope.glitch.offset
lwid = scope.glitch.width
total_successes = 0
successes = 0
resets = 0
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    #print(scope.glitch.ext_offset)
    if glitch_setting[3] == 1:
        total_successes += successes
        if (successes > 0):
            print("successes = {}, resets = {}, offset = {}, width = {}, ext_offset = {}".format(successes, resets, scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset))
            total_successes += successes
        successes = 0
        resets = 0
        #if total_successes > MAX_SUCCESSES:
        #    break
    target.flush() #Removes all data from the serial buffer.
    if scope.adc.state:#The current state of the trigger input. can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
        resets += 1

    scope.arm()

    #Do glitch loop
    target.write("g")

    ret = scope.capture()

    scope.io.vglitch_reset()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")
        resets += 1

        #Device is slow to boot?
        reboot_flush()

    else:
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=50)#For loop check
        if val['valid'] is False:
            gc.add("reset")
            reboot_flush()
            resets += 1
            #print(val)
        else:
            gcnt = struct.unpack("<I", val['payload'])[0]

            if gcnt != 2500: #for loop check
                gc.add("success")
                #print((scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                successes += 1
            else:
                gc.add("normal")

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

With that done, let's print our results. You'll likely want to ignore the "tries" and "ext_offset" parameters:

In [ ]:
results = gc.calc(ignore_params=["tries", "ext_offset"], sort="success_rate")
results

In [ ]:
gc.plot_2d(alpha=False)

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert total_successes >= 1

In [ ]:
scope.glitch.enabled = False
target.write("g0001\n")
time.sleep(0.1)
while target.in_waiting():
                data = target.read()
                print(data, end='')  # no decoding, already string

In [ ]:
scope.glitch.enabled = False
target.write("g")
time.sleep(0.1)
while target.in_waiting():
                data = target.read()
                print(data, end='')  # no decoding, already string

In [ ]:
scope.io.nrst = 'high_z'
target.write("g")

In [ ]:
    
def nrst_and_tio4_high():
    """
    Returns True if both nRST and TIO4 are high, False otherwise.

    Prereqs:
      - scope.io.nrst and scope.io.tio4 must be in 'high_z' mode (input)
      - Works on CW-Husky firmware that supports nrst_state and tio_states
    """
    # Make sure the pins are inputs (high impedance)
    scope.io.nrst = 'high_z'
    scope.io.tio4 = 'high_z'

    # Read logic levels
    nrst_high  = bool(scope.io.nrst_state)          # True if high
    tio4_high  = bool(scope.io.tio_states[3])       # TIO4 is index 3 (TIO1=0, TIO2=1, TIO3=2, TIO4=3)

    return nrst_high and tio4_high

def retry():
    scope.io.nrst = 'low'
    time.sleep(0.05)   # hold reset low
    scope.glitch.repeat = 500
    #scope.glitch.trigger_src = "manual" already manual 
    scope.arm()
    #print(scope.errors)
    scope.errors.clear()
    #scope.glitch.trigger_src = "ext_single" 
    target.flush()
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    while not nrst_and_tio4_high():
        time.sleep(0.002)
    time.sleep(0.1)   # let target start running

In [ ]:
print(nrst_and_tio4_high())

In [ ]:
while True:
    target.write("g")
    time.sleep(0.01)
    

In [ ]:
retry()

In [ ]:
while True:
        try:
            while target.in_waiting():
                data = target.read()
                print(data, end='')  # already string
        except Exception as e:
            print("UART error:", e)
        time.sleep(0.1)
        target.write("g")

In [ ]:
import serial, time

DEVICE = '/dev/ttyACM2'  # or your actual port

# Open and keep port open
s = serial.Serial(DEVICE, 115200, timeout=1)

# Optional: clear buffers
s.reset_input_buffer()
s.reset_output_buffer()

# --- later, anywhere in your notebook ---
# you can reuse s:
#s.write(b"g")
#print(s.read(1024))

# --- when you're done ---
#s.close()



In [ ]:
while True:
    ser.write(b"g")
    time.sleep(0.01)


In [ ]:
s.close()


In [ ]:
print(ser.read(1024))

In [ ]:
import serial, threading, time

DEVICE = "/dev/ttyACM2"     # set your port
BAUD   = 115200

# Open once and keep it open
ser = serial.Serial(DEVICE, BAUD, timeout=0.05)  # timeout=0 => non-blocking

stop_evt   = threading.Event()
write_lock = threading.Lock()  # prevent interleaved writes from multiple threads

def uart_reader():
    while not stop_evt.is_set():
        try:
            n = ser.in_waiting
            if n:
                chunk = ser.read(n)
                if chunk:
                    print(chunk.decode(errors="ignore"), end="")
            else:
                time.sleep(0.01)
        except serial.SerialException as e:
            print("UART read error:", e); time.sleep(0.2)

# Start the background reader (daemon so it won't block interpreter exit)
uart_thread = threading.Thread(target=uart_reader, daemon=True)
uart_thread.start()

def uart_write(data: bytes | str, add_newline=False):
    """Thread-safe write. Accepts bytes or str."""
    if isinstance(data, str):
        data = data.encode()
    if add_newline:
        data += b"\n"
    with write_lock:
        ser.write(data)
        ser.flush()  # optional

In [ ]:
while True:
    uart_write("g")
    time.sleep(0.1)


In [ ]:




    
def nrst_and_tio4_high():
    """
    Returns True if both nRST and TIO4 are high, False otherwise.

    Prereqs:
      - scope.io.nrst and scope.io.tio4 must be in 'high_z' mode (input)
      - Works on CW-Husky firmware that supports nrst_state and tio_states
    """
    # Make sure the pins are inputs (high impedance)
    scope.io.nrst = 'high_z'
    scope.io.tio4 = 'high_z'

    # Read logic levels
    nrst_high  = bool(scope.io.nrst_state)          # True if high
    tio4_high  = bool(scope.io.tio_states[3])       # TIO4 is index 3 (TIO1=0, TIO2=1, TIO3=2, TIO4=3)

    return nrst_high and tio4_high

def retry():
    scope.io.nrst = 'low'
    time.sleep(0.05)   # hold reset low
    scope.glitch.repeat = 500
    #scope.glitch.trigger_src = "manual" already manual 
    scope.arm()
    #print(scope.errors)
    scope.errors.clear()
    #scope.glitch.trigger_src = "ext_single" 
    target.flush()
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    #
    while not nrst_and_tio4_high(): 
        time.sleep(0.002)
    time.sleep(0.05)   # let target start running


gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


scope.glitch.trigger_src = "manual"

num_tries = 20 # increase to get better glitch stats
gc.set_global_step([2]) # reduce to fine tune glitching
gc.set_range("ext_offset", 5000, 5050)
gc.set_range("repeat", 98, 105)
gc.set_range("tries", 1, num_tries)
gc.set_step("repeat", 1)
gc.set_step("ext_offset", 1)
gc.set_step("tries", 1)
scope.glitch.enabled = True



for glitch_setting in gc.glitch_values():
    
    retry()
    
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    scope.arm()#trigger a glitch from python
    #time.sleep(0.1)
    val = target.read(5, 2)
    
    glitch = (val == "r0000")
    
    if glitch is True:
        gc.add("success")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "YAYYYYY"))
        
    elif nrst_and_tio4_high():
        gc.add("normal")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "normal"))
        
    else:
        gc.add("reset")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val, "reset"))
        #reboot_flush()

print("Done glitching")

        

In [ ]:
def reboot_flush(expected_msg="s", timeout=10.0):
    scope.io.nrst = 'low'
    time.sleep(0.05)   # hold reset low
    target.flush()
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)   # let target start running
    
    start_time = time.time()
    buffer = ""

    while time.time() - start_time < timeout:
        data = target.read(100, timeout=0.01)
        if data:
            buffer += data
            if expected_msg in buffer:
                #print(f"[reboot_flush] Detected startup message: '{expected_msg}'")
                return True
        time.sleep(0.01)

    #print(f"[reboot_flush] Timeout waiting for '{expected_msg}' (got: '{buffer.strip()}')")
    return False

reboot_flush()

In [ ]:
target.write("g")

In [ ]:
is_high = bool(scope.io.tio_states[3])
print("Trigger HIGH?", is_high)

In [ ]:
scope.errors.clear()